In [34]:
import pandas as pd
import numpy as np

In [35]:
oil_df = pd.read_csv('oil_consumption_per_capita.csv')
mortality_df = pd.read_csv('mortality_rate.csv')
population_df = pd.read_csv('population.csv')
gdp_df = pd.read_csv('gdp_per_capita_usdollars.csv')

In [36]:
mortality_df.drop(columns=['Indicator Name'], inplace=True)
mortality_df.rename(columns={'Country Name': 'Country'}, inplace=True)
oil_df.rename(columns={'Oil Consumption per capita (tonnes per year)': 'Country'}, inplace=True)
population_df.rename(columns={'Country Name': 'Country'}, inplace=True)
gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)

In [37]:
mortality_melted = mortality_df.melt(id_vars=['Country'], var_name='Year', value_name='Mortality Rate')
oil_melted = oil_df.melt(id_vars=['Country'], var_name='Year', value_name='Oil Consumption per capita (tonnes per year)')
population_melted = population_df.melt(id_vars=['Country'], var_name='Year', value_name='Population')
gdp_melted = gdp_df.melt(id_vars=['Country'], var_name='Year', value_name='GDP per capita (US$)')

In [38]:
merged_df = pd.merge(mortality_melted, oil_melted, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, population_melted, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, gdp_melted, on=['Country', 'Year'], how='inner')

In [39]:
knn_df = merged_df.drop(columns=['Country'])

In [40]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputed_df = pd.DataFrame(imputer.fit_transform(knn_df), columns=knn_df.columns)

In [41]:
merged_df.update(imputed_df)

In [42]:
merged_df['Year'] = merged_df['Year'].astype(int)
merged_df['Mortality Rate'] = round(merged_df['Mortality Rate'], 1)
merged_df['Oil Consumption per capita (tonnes per year)'] = round(merged_df['Oil Consumption per capita (tonnes per year)'], 2)
merged_df['Population'] = merged_df['Population'].astype(int)
merged_df['GDP per capita (US$)'] = merged_df['GDP per capita (US$)'].astype(int)

In [43]:
country_to_continent = {
    'United States': 'North America',
    'Canada': 'North America',
    'Mexico': 'North America',
    'Argentina': 'South America',
    'Ecuador': 'South America',
    'Brazil': 'South America',
    'Chile': 'South America',
    'Colombia': 'South America',
    'Peru': 'South America',
    'Azerbaijan': 'Asia',
    'Bangladesh': 'Asia',
    'China': 'Asia',
    'United Arab Emirates': 'Asia',
    'Saudi Arabia': 'Asia',
    'India': 'Asia',
    'Indonesia': 'Asia',
    'Russia': 'Asia',
    'Japan': 'Asia',
    'Kazakhstan': 'Asia',
    'Korea, Rep.': 'Asia',
    'Kuwait': 'Asia',
    'Malaysia': 'Asia',
    'Pakistan': 'Asia',
    'Philippines': 'Asia',
    'Qatar': 'Asia',
    'Singapore': 'Asia',
    'Thailand': 'Asia',
    'Turkmenistan': 'Asia',
    'Uzbekistan': 'Asia',
    'Austria': 'Europe',
    'Bulgaria': 'Europe',
    'Belarus': 'Europe',
    'Switzerland': 'Europe',
    'United Kingdom': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'Spain': 'Europe',
    'Denmark': 'Europe',
    'Finland': 'Europe',
    'Greece': 'Europe',
    'Hungary': 'Europe',
    'Ireland': 'Europe',
    'Iceland': 'Europe',
    'Italy': 'Europe',
    'Lithuania': 'Europe',
    'Netherlands': 'Europe',
    'Norway': 'Europe',
    'Poland': 'Europe',
    'Portugal': 'Europe',
    'Romania': 'Europe',
    'Slovak Republic': 'Europe',
    'Ukraine': 'Europe',
    'Sweden': 'Europe',
    'Australia': 'Oceania',
    'New Zealand': 'Oceania',
    'South Africa': 'Africa',
    'Algeria': 'Africa',
}

merged_df['Continent'] = merged_df['Country'].apply(lambda x: country_to_continent.get(x, 'Other'))

In [44]:
merged_df.loc[:, 'GDP per capita (log US$)'] = merged_df['GDP per capita (US$)'].apply(lambda x: np.log(x))

In [45]:
merged_df

,Country,Year,Mortality Rate,Oil Consumption per capita (tonnes per year),Population,GDP per capita (US$),Continent,GDP per capita (log US$)
0,United Arab Emirates,1965,149.7,1.77,173797,7412,Asia,8.910856
1,Argentina,1965,32.6,0.98,22053661,1285,South America,7.158514
2,Australia,1965,22.4,1.49,11388000,2281,Oceania,7.732369
3,Austria,1965,32.7,0.76,7270889,1374,Europe,7.225481
4,Azerbaijan,1965,23.3,3.21,4574650,18842,Asia,9.843844
...,...,...,...,...,...,...,...,...
2571,Turkmenistan,2010,43.2,1.11,5267970,4286,Asia,8.363109
2572,Ukraine,2010,11.7,0.26,45870741,3078,Europe,8.032035
2573,United States,2010,7.3,2.74,309327143,48650,North America,10.792407
2574,Uzbekistan,2010,28.0,0.18,28562400,1742,Asia,7.462789


In [46]:
merged_df.to_csv('oil_consumption_mortality.csv', index=False)